# STUDIO ESECUTIVO E FINALE

#### Maraziti Matteo, Tocci Federico e Scordino Giacomo

[Link Repository](https://github.com/FTocci/TGW2D)

Di seguito è illustrato il codice eseguito prima di effettuare il Profiling dell'algoritmo e la valutazione sulle prestazioni.

##### NB: Prima di eseguire il Profiling è necessario eseguire le istruzioni nelle successive due finestre.

In [23]:
using SparseArrays
using ViewerGL
GL = ViewerGL
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools
include("../src/arrangement/planar_arrangement.jl")

planar_arrangement

In [ ]:
# input generation
V,EV = Lar.randomcuboids(100, .30)
V = GL.normalize2(V,flag=true)
VV = [[k] for k=1:size(V,2)]
#GL.VIEW( GL.numbering(.05)((V,[VV, EV]),GL.COLORS[1]) )

# subdivision of input edges
W = convert(Lar.Points, V')
cop_EV = Lar.coboundary_0(EV::Lar.Cells)
cop_EW = convert(Lar.ChainOp, cop_EV);

Dalle premesse poste in essere all'interno del Notebook Preliminare, il primo passo da affrontare è stato quello di analizzare nel dettaglio dove si presentassero eventuali "colli di bottiglia" che peggioravano le prestazioni.

Mediante l'impiego del Profiling è possibile identificarli.

In [22]:
using Profile
@profile planar_arrangement(W::Lar.Points, cop_EW::Lar.ChainOp)
Profile.print(format=:flat)

 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
     3         3 @Base\Base.jl             35 getproperty(x::Module, f::Symbol)
     5         5 @Base\Base.jl             42 getproperty
     7         7 @Base\Base.jl             42 getproperty(x::SparseVector{Int...
   768         0 @Base\Base.jl            418 include(mod::Module, _path::Str...
     1         1 @Base\Base.jl             43 setproperty!
     2         0 @Base\abstractarray.jl  2760 mapslices(f::LinearAlgebraicRep...
     1         0 @Base\abstractarray.jl  2770 mapslices(f::LinearAlgebraicRep...
     3         3 @Base\abstractarray.jl  2775 mapslices(f::LinearAlgebraicRep...
     3         3 @Base\abstractarray.jl  2797 mapslices(f::LinearAlgebraicRep...
     1         0 @Base\abstractarray.jl  2802 mapslices(f::LinearAlgebraicRep...
     1         1 @Base\abstractarray.jl  2805 mapslices(f::LinearAlgebraicRep...
    13         1 @Base\abstractarray.jl  2806

   661 delete!(h::Dict{Int64, Nothing}...
    44         2 @Base\dict.jl            662 delete!(h::Dict{Int64, Nothing}...
     1         0 @Base\dict.jl            663 delete!(h::Dict{Int64, Nothing}...
     3         3 @Base\dict.jl            666 delete!(h::Dict{Int64, Nothing}...
     1         1 @Base\dict.jl            479 getindex(h::Dict{Float64, Int64...
     4         4 @Base\dict.jl            280 getproperty
    20         0 @Base\dict.jl            169 hashindex
     1         1 @Base\dict.jl            169 hashindex(key::Int64, sz::Int64)
    55         0 @Base\dict.jl            552 haskey
     2         2 @Base\dict.jl              ? ht_keyindex(h::Dict{Any, Nothin...
     8         8 @Base\dict.jl            280 ht_keyindex(h::Dict{Any, Nothin...
     6         0 @Base\dict.jl            281 ht_keyindex(h::Dict{Int64, Noth...
     3         0 @Base\dict.jl            283 ht_keyindex(h::Dict{Any, Nothin...
    17         0 @Base\dict.jl            284 ht_keyindex(h::Dic

     7 (::LinearAlgebraicRepresentatio...
     3         0 ...t\minimal_cycles.jl     8 (::LinearAlgebraicRepresentatio...
     8         0 ...t\minimal_cycles.jl    14 minimal_2cycles(V::Matrix{Float...
     3         3 ...t\minimal_cycles.jl    15 minimal_2cycles(V::Matrix{Float...
   117         0 ...t\minimal_cycles.jl    18 minimal_2cycles(V::Matrix{Float...
     4         0 ...anar_arrangement.jl    62 (::LinearAlgebraicRepresentatio...
     1         0 ...anar_arrangement.jl    63 (::LinearAlgebraicRepresentatio...
    65         0 ...anar_arrangement.jl   173 an_edge
    65         0 ...anar_arrangement.jl   200 biconnected_components(EV::Spar...
     1         0 ...anar_arrangement.jl   202 biconnected_components(EV::Spar...
    28         0 ...anar_arrangement.jl   203 biconnected_components(EV::Spar...
     1         0 ...anar_arrangement.jl   215 biconnected_components(EV::Spar...
     3         0 ...anar_arrangement.jl   221 biconnected_components(EV::Spar...
     1       

    19         1 ...src\sparsevector.jl   603 getindex(A::SparseMatrixCSC{Int...
     8         0 ...src\sparsevector.jl  1097 hcat(::SparseMatrixCSC{Int64, I...
     8         0 ...src\sparsevector.jl  1098 hcat(::SparseMatrixCSC{Int64, I...
     1         0 ...src\sparsevector.jl  1710 mul!
     1         0 ...src\sparsevector.jl  1719 mul!(y::Vector{Int64}, A::Spars...
     2         0 ...src\sparsevector.jl   138 spzeros
     2         0 ...src\sparsevector.jl   139 spzeros
     1         0 ...src\sparsevector.jl  1130 vcat(::Matrix{Float64}, ::Matri...
    17         0 ...src\sparsevector.jl  1130 vcat
   768         0 ...otebook\notebook.jl    32 top-level scope
   768         0 @JSONRPC\src\typed.jl     67 dispatch_msg(x::VSCodeServer.JS...
   768         0 ...eServer\src\repl.jl   219 withpath(f::VSCodeServer.var"#1...
   768         0 ...c\serve_notebook.jl    19 (::VSCodeServer.var"#192#193"{V...
   768         0 ...c\serve_notebook.jl   139 serve_notebook(pipename::String...

-------------------------------------

Dal Profiling è stato possibile individuare che, come già intuito nel Notebook Preliminare, le chiamate a funzioni esterne a planar_arrangement provocano un innalzamento dei tempi di esecuzione.

Le chiamate più onerose sono quelle dove vi è un valore 'count' più elevato.

### ! NB: La misurazione delle tempistiche è stata effettuata su linea di comando Julia, non all'interno del Notebook. Di seguito sono riportate le istruzioni eseguite

```julia
    using SparseArrays
    using ViewerGL
    GL = ViewerGL
    using LinearAlgebraicRepresentation
    Lar = LinearAlgebraicRepresentation
    include("../src/arrangement/planar_arrangement.jl")     #IL PATH VARIA LOCALMENTE
                                                            #QUINDI VA MODIFICATO
    
    V,EV = Lar.randomcuboids(100, .30)
    V = GL.normalize2(V,flag=true)
    VV = [[k] for k=1:size(V,2)]
    W = convert(Lar.Points, V')
    cop_EV = Lar.coboundary_0(EV::Lar.Cells)
    cop_EW = convert(Lar.ChainOp, cop_EV);
    
    @time planar_arrangement(W::Lar.Points, cop_EW::Lar.ChainOp)
```

##### Risultato test: 0.976047 seconds (9.80 M allocations: 522.859 MiB, 6.45% gc time)

## Soluzione adottata per incrementare le prestazioni

Per migliorare le tempistiche è stata implementata la tecnica del MultiThreading mediante l'utilizzo della macro *Threads.@threads*. Essa è stata inserita così da poter eseguire in parallelo le istruzioni cicliche presenti nel codice.

Di seguito vi sono riportate alcune delle porzioni di codice all'interno delle quali è stata implementata la macro:

```julia
function cleandecomposition(V, copEV, sigma, edge_map)
    ...
    Threads.@threads for e in 1:copEV.m
        if !(e in new_edges)

            vidxs = copEV[e, :].nzind
            v1, v2 = map(i->V[vidxs[i], :], [1,2])
            centroid = .5*(v1 + v2)

            if ! Lar.point_in_face(centroid, V, ev)
                push!(todel, e)
            end
        end
    end

    Threads.@threads for i in reverse(todel)
        for row in edge_map

            filter!(x->x!=i, row)

            for j in 1:length(row)
                if row[j] > i
                    row[j] -= 1
                end
            end
        end
    end

   ...
end
```

```julia
function get_external_cycle(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp)
    ...
    Threads.@threads for i in vs
        if V[i, 1] > V[maxv_x1, 1]
            maxv_x1 = i
        elseif V[i, 1] < V[minv_x1, 1]
            minv_x1 = i
        end
        if V[i, 2] > V[maxv_x2, 2]
            maxv_x2 = i
        elseif V[i, 2] < V[minv_x2, 2]
            minv_x2 = i
        end
    end
    ...
end
```

```julia
    function componentgraph(V, copEV, bicon_comps)
    ...
	Threads.@threads for i in 1:n
    	vs_indexes = (abs.(EVs[i]')*abs.(shells[i])).nzind
   		push!(shell_bboxes, Lar.bbox(V[vs_indexes, :]))
	end
    ...
end
```

Grazie all'implementazione della macro è possibile eseguire le istruzioni su più thread concorrenti. 

Per modificare il numero di Thread(su Windows) nell'enviroment Julia si utilizza l'istruzione *julia --threads n*, dove n rappresenta il numero di thread prescelti.

I successivi test sono stati eseguiti con 8 threads, portando i seguenti risultati:

##### ! NB: Le istruzioni sono state eseguite anche in questo caso su linea di comando Julia e sono riportate di seguito

```julia
    using SparseArrays
    using ViewerGL
    GL = ViewerGL
    using LinearAlgebraicRepresentation
    Lar = LinearAlgebraicRepresentation
    include("../src/arrangement/planar_arrangement.jl") #IL PATH VARIA LOCALMENTE QUINDI VA MODIFICATO
    
    V,EV = Lar.randomcuboids(100, .30)
    V = GL.normalize2(V,flag=true)
    VV = [[k] for k=1:size(V,2)]
    W = convert(Lar.Points, V')
    cop_EV = Lar.coboundary_0(EV::Lar.Cells)
    cop_EW = convert(Lar.ChainOp, cop_EV);
    
    @time planar_arrangement(W::Lar.Points, cop_EW::Lar.ChainOp)
```


##### Risultato test:  0.839191 seconds (8.70 M allocations: 452.370 MiB, 5.68% gc time)

Le modifiche effettuate nel codice hanno portato dei benefici in termini di prestazioni, riducendo il tempo di esecuzione dell'algoritmo del 15% circa  

### Impiego della macchina DGX